<a href="https://colab.research.google.com/github/Krukalex/First-repo/blob/main/Bike_Demand_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install kaggle

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c bike-sharing-demand

In [ ]:
!unzip bike-sharing-demand.zip

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
import seaborn as sns

In [ ]:
test=pd.read_csv('test.csv')
train=pd.read_csv('train.csv')

##Decision Tree

In [ ]:
train.head()

In [ ]:
train.dtypes

#Exploratory analysis

In [ ]:
exp=train.copy()

##Demand by hour

In [ ]:
#convert date info to date time and make it index

dt = pd.DatetimeIndex(exp['datetime'])
exp.set_index(dt, inplace=True)

In [ ]:
#make seperate columns in data for different time denoms

exp['date'] = dt.date
exp['day'] = dt.day
exp['month'] = dt.month
exp['year'] = dt.year
exp['hour'] = dt.hour
exp['dow'] = dt.dayofweek
exp['woy'] = dt.weekofyear

In [ ]:
exp.head()

In [ ]:
#see how demand fluctuates over the hours of a day depending on whether or not it is a work day
#spikes midday on weekends and during communting times on working days

plt.bar(exp['hour'].unique(), exp[exp['workingday']==1].groupby('hour')['count'].sum(), color='r')
plt.bar(exp['hour'].unique(), exp[exp['workingday']==0].groupby('hour')['count'].sum(), color='b')


##Daily trend

is there a difference in demand between registered and casual users depending on the day of the week?

In [ ]:
exp.groupby('dow')['casual'].sum()

In [ ]:
exp.groupby('dow')['registered'].sum()

In [ ]:
plt.bar(exp['dow'].unique(), exp.groupby('dow')['casual'].sum(), color='r', width=.25)
plt.bar(exp['dow'].unique()+.25, exp.groupby('dow')['registered'].sum(), color='b', width=.25)
plt.show()

##Weather

In [ ]:
plt.bar(exp['weather'].unique(),exp.groupby('weather')['count'].sum())

In [ ]:
exp[exp['weather']==1][['hour', 'season']]

good weather

In [ ]:
season_map = {1:'Spring', 2:'Summer', 3:'Fall', 4:'Winter'}

In [ ]:
good_weather=exp[exp['weather']==1][['hour', 'season']].copy()
data=pd.DataFrame({'count' : good_weather.groupby(["hour","season"]).size()}).reset_index()
data['season'] = data['season'].map(lambda d : season_map[d])

In [ ]:
fig, ax = plt.subplots(figsize=(18, 5))
sns.pointplot(data['hour'], data['count'], hue=data['season'])
ax.set(xlabel='Hour Of The Day', ylabel='Good Weather Count', title="Good Weather By Hour Of The Day Across Season");

normal weather

In [ ]:
normal_weather=exp[exp['weather']==2][['hour', 'season']].copy()
data2=pd.DataFrame({'count' : normal_weather.groupby(["hour","season"]).size()}).reset_index()
data2['season'] = data2['season'].map(lambda d : season_map[d])

In [ ]:
fig, ax = plt.subplots(figsize=(18, 5))
sns.pointplot(data2['hour'], data2['count'], hue=data2['season'])
ax.set(xlabel='Hour Of The Day', ylabel='Normal Weather Count', title="Normal Weather By Hour Of The Day Across Season");

Bad weather

In [ ]:
bad_weather=exp[exp['weather']==3][['hour', 'season']].copy()
data2=pd.DataFrame({'count' : bad_weather.groupby(["hour","season"]).size()}).reset_index()
data2['season'] = data2['season'].map(lambda d : season_map[d])

In [ ]:
fig, ax = plt.subplots(figsize=(18, 5))
sns.pointplot(data2['hour'], data2['count'], hue=data2['season'])
ax.set(xlabel='Hour Of The Day', ylabel='Bad Weather Count', title="Bad Weather By Hour Of The Day Across Season");

Renting by hour based on weather

In [ ]:
weather_map = {1:'Good', 2:'Normal', 3:'Bad', 4:'Worse'}
renting=pd.DataFrame(exp.groupby(['weather', 'hour'],sort=True)['count'].mean()).reset_index()
renting['weather']=renting['weather'].map(lambda d: weather_map[d])

In [ ]:
renting.head()

In [ ]:
fig, ax = plt.subplots(figsize=(18, 5))
sns.pointplot(renting['hour'], renting['count'], hue=renting['weather'])
ax.set(xlabel='Hour Of The Day', ylabel='Count', title="Renting by weather across hours");

##seasons

Renting by hour across seasons

In [ ]:
data=pd.DataFrame(exp.groupby(['season', 'hour'],sort=True)['count'].mean()).reset_index()
data['season']=data['season'].map(lambda d: season_map[d])

In [ ]:
fig, ax = plt.subplots(figsize=(18, 5))
sns.pointplot(data['hour'], data['count'], hue=data['season'])
ax.set(xlabel='Hour Of The Day', ylabel='Count', title="Renting by season across hours");

##Days of week

timing of renting depending on day of week

In [ ]:
day_map = {0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}

In [ ]:
data=pd.DataFrame(exp.groupby(['dow', 'hour'])['count'].mean()).reset_index()
data['dow']=data['dow'].map(lambda d: day_map[d])

In [ ]:
#more bikes are rented in mornings and evenings on weekdays and more are rented around midday on weekends

fig, ax = plt.subplots(figsize=(18, 5))
sns.pointplot(data['hour'], data['count'], hue=data['dow'])
ax.set(xlabel='Hour Of The Day', ylabel='Count', title="Renting by hour across days");

How do renting habits differ between casual and registered bikers over the course of a day?

In [ ]:
data=pd.DataFrame(pd.melt(exp[["hour","casual","registered"]], id_vars=['hour'], value_vars=['casual', 'registered'], var_name='usertype', value_name='count').groupby(['usertype', 'hour'])['count'].mean()).reset_index()

In [ ]:
fig, ax = plt.subplots(figsize=(18, 5))
sns.pointplot(data['hour'], data['count'], hue=data['usertype'])
ax.set(xlabel='Hour of day', ylabel='Count', title='Daily renting volume of casual vs registered bikers')

In [ ]:
#significantly different trends over the course of the day between caual and registered buyers

fig, axs = plt.subplots(1, 2, figsize=(18,5), sharex=False, sharey=False)

sns.boxplot(x='hour', y='casual', data=exp, ax=axs[0])
axs[0].set_ylabel('casual users')
axs[0].set_title('')

sns.boxplot(x='hour', y='registered', data=exp, ax=axs[1])
axs[1].set_ylabel('registered users')
axs[1].set_title('');

In [ ]:
#count variable is right skewed, and a log trasnformation makes it more normal
plt.hist(exp['count'])
plt.show()
plt.hist(np.log(exp['count']))
plt.show()

In [ ]:
exp=exp.assign(log_count=lambda x:np.log(exp['count']) )

In [ ]:
#if you plot the log count by hour, there seems to be fewer outliers than there were with normal count

fig, ax = plt.subplots(figsize=(18, 5))
sns.boxplot(x='hour', y='log_count', data=exp, ax=ax)
ax.set(ylabel='log(count) of Users',title='Boxplot of Log of Count grouped by hour')

In [ ]:
# logarithmic transformation of dependent cols
# (adding 1 first so that 0 values don't become -inf)
for col in ['casual', 'registered', 'count']:
    exp['%s_log' % col] = np.log(exp[col] + 1)

##More rentals on workdays when mornings/evenings are warm?

graphing a categorical variable against a continuous one, which is why we use the jitter function to make it look a bit cleaner on the graph

In [ ]:
def hour_jitter(h):
    #return h + ((np.random.randint(low=0, high=9, size=1)[0] - 4) / 10)
    return h + np.random.uniform(-0.4, 0.4)

In [ ]:
def hour_format(h):
    return "{:02d}:00 AM".format(h) if h <= 12 else "{:02d}:00 PM".format(h%12)

In [ ]:
# jitter plot
import matplotlib.colors as mcolors
import matplotlib.cm as cm

# color_map = plt.get_cmap("jet")
color_map = mcolors.ListedColormap(list(["#5e4fa2", "#3288bd", "#66c2a5", "#abdda4", "#e6f598", "#fee08b", "#fdae61", "#f46d43", "#d53e4f", "#9e0142"]))
exp['hour_jitter'] = exp['hour'].map(hour_jitter)


In [ ]:
#warmer mornings and evenings seem to have higher rental rates

exp[exp['workingday']==1].plot(kind='scatter', x='hour_jitter', y='count', figsize=(18,6), c='temp', cmap=color_map, colorbar=True, sharex=False)
hours = np.unique(exp['hour'].values)
hour_labels = [hour_format(h) for h in hours]
plt.xticks(hours, hour_labels, rotation='vertical');

In [ ]:
exp.drop('hour_jitter', axis=1, inplace=True);

In [ ]:
#once again looking at how registered users are more active during the week and casual users are more active on weekends

dayOfWeek={0:'Monday', 1:'Tuesday', 2:'Wednesday', 3:'Thursday', 4:'Friday', 5:'Saturday', 6:'Sunday'}
exp['weekday'] = exp['dow'].map(dayOfWeek)

fig, axs = plt.subplots(1, 2, figsize=(15,5), sharex=False, sharey=False)

sns.boxplot(x='weekday', y='registered', data=exp, ax=axs[0])
axs[0].set_ylabel('registered users')
axs[0].set_title('')

sns.boxplot(x='weekday', y='casual', data=exp, ax=axs[1])
axs[1].set_ylabel('casual users')
axs[1].set_title('')

exp.drop('weekday', axis=1, inplace=True);

In [ ]:
#confirming that bad weather substantially reduces bike rentals

fig, axs = plt.subplots(1, 2, figsize=(15,5), sharex=False, sharey=False)

sns.boxplot('weather', 'registered', data=exp, ax=axs[0])
sns.boxplot('weather', 'casual', data=exp, ax=axs[1])

##looking at things like humidity, windspeed, airtemp and temperature

In [ ]:
sub_df = exp[['count', 'registered', 'casual', 'temp', 'atemp', 'humidity', 'windspeed', 'workingday', 'holiday']]

In [ ]:
#correlation matrix

sub_df.corr()

In [ ]:
#make it prettier
#strong correlation between temp and airtemp
#temp correlates with rental counts of both registered and casual riders
#windspeed has relatively low correlation

corrMatt = sub_df.corr()
mask = np.zeros_like(corrMatt)
mask[np.triu_indices_from(mask)] = True
fig, ax = plt.subplots(figsize=(15, 6))
sns.heatmap(corrMatt, mask=mask, vmax=.8, square=False, annot=True, ax=ax, linewidths=1);

In [ ]:
exp['season'].reset_index()

##Time

In [ ]:
season_map = {1:'Spring', 2:'Summer', 3:'Fall', 4:'Winter'}
data = exp['season'].reset_index().copy()
data['season'] = data['season'].map(lambda d : season_map[d])

data2 = test['season'].reset_index().copy()
data2['season'] = data2['season'].map(lambda d : season_map[d])

In [ ]:
plt.bar(exp['season'].unique(),exp.groupby('season')['count'].sum())


How does the data differ over the two year horizon

In [ ]:
plt.figure(figsize=(8, 5))
sns.boxplot(x='year', y='count', data=exp)
plt.ylabel('Count of Users')
plt.title("Boxplot of Count grouped by year");

#Model selection

In [ ]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

In [ ]:
def get_rmsle(y_pred, y_actual):
    diff = np.log(y_pred + 1) - np.log(y_actual + 1)
    mean_error = np.square(diff).mean()
    return np.sqrt(mean_error)

##Try running models before adding or modifying features

###Linear Regression

In [ ]:
from sklearn import datasets, linear_model, metrics

In [ ]:
reg = linear_model.LinearRegression()

In [ ]:
#without feature engineering, can't use time stamp, so select every column after that and before the count columns as predictors
#for this initial attempt, I am only going to try to predict the total count, and disregard casual vs registered info all together

X=train.iloc[:,1:-3]
y=train.iloc[:,-1:]

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=42, test_size=0.3)

In [ ]:
reg.fit(X_train,y_train)

In [ ]:
#here are the coefficients of the model

print('Coefficients: ', reg.coef_)

In [ ]:
pred=reg.predict(X_valid)

In [ ]:
pred

In [ ]:
#pretty big rmse

rmse(pred, y_valid)

In [ ]:
#rmsle is pretty far from best models, which get around 0.3

get_rmsle(pred, y_valid)

###Random Forest Regression

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
dt_reg= RandomForestRegressor(n_estimators = 100, random_state = 24)

In [ ]:
dt_reg.fit(X_train, y_train)

In [ ]:
dt_pred=dt_reg.predict(X_valid)

In [ ]:
dt_pred=dt_pred.reshape((3266,1))

In [ ]:
#slightly improved over linear regression

rmse(dt_pred, y_valid)

In [ ]:
#also slight improvement but still subpar

get_rmsle(dt_pred, y_valid)

###Neural Network

In [ ]:
X_train.shape

In [ ]:
# Installing required libraries
!pip install tensorflow
!pip install keras


In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import SGD

In [ ]:
from sklearn.preprocessing import StandardScaler
PredictorScaler=StandardScaler()
TargetVarScaler=StandardScaler()

In [ ]:
PredictorScalerFit=PredictorScaler.fit(X)
TargetVarScalerFit=TargetVarScaler.fit(y)

In [ ]:
X_t=PredictorScalerFit.transform(X)
y_t=TargetVarScalerFit.transform(y)

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_t, y_t, random_state=42, test_size=0.3)

In [ ]:
model = Sequential()

In [ ]:
model.add(Dense(units=5, input_dim=8, kernel_initializer='normal', activation='relu'))
model.add(Dense(units=5, kernel_initializer='normal', activation='tanh'))
model.add(Dense(1, kernel_initializer='normal'))
model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
model.fit(X_train, y_train ,batch_size = 20, epochs = 50, verbose=1)

In [ ]:
def FunctionFindBestParams(X_train, y_train, X_test, y_test):
    
    # Defining the list of hyper parameters to try
    batch_size_list=[5, 10, 15, 20]
    epoch_list  =   [5, 10, 50, 100]
    
    import pandas as pd
    SearchResultsData=pd.DataFrame(columns=['TrialNumber', 'Parameters', 'Accuracy'])
    
    # initializing the trials
    TrialNumber=0
    for batch_size_trial in batch_size_list:
        for epochs_trial in epoch_list:
            TrialNumber+=1
            # create ANN model
            model = Sequential()
            # Defining the first layer of the model
            model.add(Dense(units=5, input_dim=X_train.shape[1], kernel_initializer='normal', activation='relu'))
 
            # Defining the Second layer of the model
            model.add(Dense(units=5, kernel_initializer='normal', activation='relu'))
 
            # The output neuron is a single fully connected node 
            # Since we will be predicting a single number
            model.add(Dense(1, kernel_initializer='normal'))
 
            # Compiling the model
            model.compile(loss='mean_squared_logarithmic_error', optimizer=opt, metrics=['mse'])
 
            # Fitting the ANN to the Training set
            model.fit(X_train, y_train ,batch_size = batch_size_trial, epochs = epochs_trial, verbose=0)

            RMSLE=get_rmsle(model.predict(X_test), y_test)
 
            MAPE = np.mean(100 * (np.abs(y_test-model.predict(X_test))/y_test))
            
            # printing the results of the current iteration
            print(TrialNumber, 'Parameters:','batch_size:', batch_size_trial,'-', 'epochs:',epochs_trial, 'Accuracy:', RMSLE)
            
            SearchResultsData=SearchResultsData.append(pd.DataFrame(data=[[TrialNumber, str(batch_size_trial)+'-'+str(epochs_trial), 100-MAPE]],
                                                                    columns=['TrialNumber', 'Parameters', 'Accuracy'] ))
    return(SearchResultsData)
 

In [ ]:
##batch size of 10 and 10 epochs is best

ResultsData=FunctionFindBestParams(X_train, y_train, X_valid, y_valid)

In [ ]:
%matplotlib inline
ResultsData.plot(x='Parameters', y='Accuracy', figsize=(15,4), kind='line')

In [ ]:
model.fit(X_train, y_train ,batch_size = 15, epochs = 50, verbose=1)

In [ ]:
Predictions=model.predict(X_valid)

In [ ]:
Predictions=TargetVarScalerFit.inverse_transform(Predictions)

In [ ]:
y_valid_orig=TargetVarScalerFit.inverse_transform(y_valid)

In [ ]:
#this is slight improvement over random forest

rmse(Predictions, y_valid_orig)

In [ ]:
#this is somehow worse than random forest

get_rmsle(Predictions, y_valid_orig)

Results of various models are pretty mediocre without implementing some feature engineering

##Feature engineering

Exp dataframe has been modified to contain things like the hour, day and month as well as the log counts of number of bikes used. It also has datetime in a useable formant

In [ ]:
exp

Lets try running the test with the features we have already engineered into exp

In [ ]:
features=['weather', 'temp', 'atemp', 'windspeed',
    'workingday', 'season', 'holiday',
    'hour', 'dow', 'woy']
exp[features]

In [ ]:
X=exp[features]
y=exp['count']

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=42, test_size=0.3)

In [ ]:
reg_rf=RandomForestRegressor(n_estimators = 100, random_state = 24)

In [ ]:
reg_rf.fit(X_train, y_train)

In [ ]:
preds=reg_rf.predict(X_valid)

In [ ]:
rmse(preds, y_valid)

In [ ]:
get_rmsle(preds, y_valid)

Results are far better than they were before, and even starting to approach state of the art levels

We will add some additonal features and refine our testing/training split

In [ ]:
#add a binary categorical variable peak that is 1 if it is a working day and it is either 8 am or between 17 and 18 pm or if it is a non working day between 10:00 and 19:00

exp['peak'] = exp[['hour', 'workingday']]\
    .apply(lambda df: 1 if ((df['workingday'] == 1 and (df['hour'] == 8 or 17 <= df['hour'] <= 18)) \
                            or (df['workingday'] == 0 and 10 <= df['workingday'] <= 19)) else 0, axis = 1)

In [ ]:
# from histogram
#add two more variable, ideal and sticking
#ideal is one if it is at least 27 celcius and the windspeed is less than 0
#sticky is one if it is a working day and the humidity is above 60

exp['ideal'] = exp[['temp', 'windspeed']]\
    .apply(lambda df: 1 if (df['temp'] > 27 and df['windspeed'] < 30) else 0, axis = 1)
    
exp['sticky'] = exp[['humidity', 'workingday']]\
    .apply(lambda df: 1 if (df['workingday'] == 1 and df['humidity'] >= 60) else 0, axis = 1)

##Training model

In [ ]:
#we can't split using the holdout method since we want to predict the last few days of the month using the days previous
#the test wants to use the first 20 days to predict the last 10, so for validation we will use the first 15 days as training and the next 4 as validation

def custom_valid_split(data, cutoff=15):
  train=data[data['day']<=cutoff]
  valid=data[data['day']>cutoff]

  return train, valid

In [ ]:
def prep_train_data(data, input_cols):
  X=data[input_cols].values
  y_cl=data['casual_log']
  y_rl=data['registered_log']

  return X, y_cl, y_rl

###Random Forest

In [ ]:
features=[ 'weather', 'temp', 'atemp', 'windspeed',
    'workingday', 'season', 'holiday', 'sticky',
    'hour', 'dow', 'woy', 'peak']

In [ ]:
train_data, valid_data=custom_valid_split(exp)

In [ ]:
X_train,y_cl_train, y_rl_train=prep_train_data(train_data, features)
X_val,y_cl_val, y_rl_val=prep_train_data(valid_data, features)

In [ ]:
reg_rf_c=RandomForestRegressor(n_estimators = 100, random_state = 24)

In [ ]:
reg_rf_c.fit(X_train, y_cl_train)

RandomForestRegressor(random_state=24)

In [ ]:
#convert it back from log scale

preds_c=np.exp(reg_rf_c.predict(X_val))-1

In [ ]:
reg_rf_r=RandomForestRegressor(n_estimators = 100, random_state = 24)

In [ ]:
reg_rf_r.fit(X_train, y_rl_train)

RandomForestRegressor(random_state=24)

In [ ]:
preds_r=np.exp(reg_rf_r.predict(X_val))-1

In [ ]:
y_pred_comb = np.round(preds_r + preds_c)
y_pred_comb[y_pred_comb < 0] = 0

In [ ]:
y_actual_comb = np.exp(y_cl_val) + np.exp(y_rl_val) - 2

####Results

In [ ]:
get_rmsle(y_pred_comb, y_actual_comb)

0.44011142666862885

In [ ]:
rmse(y_pred_comb, y_actual_comb)

84.45066687485688

Interestingly, after all of that, splitting up the casual and registered users and predicting thier log values seperately actually didnt improve the model. this could however be attributed to the fact that in the previous random forest model, we split using the holdout method, which is not what the problem calls for, so this may have provided overyly optimistic results

###Neural Network

since we need to train the model using transformed training data and test it using transformed validation data, we will need to split the model into training and validation first, since the date information that we are splitting on will be lost in the transformation. Then, we will need to transform the training data and then validation data seperately. 

Our method is also to predict the casual riders and registed riders seperately so we will need to train two neural networks, one for each type of user, optimize them seperately, and then combine the predictions and compare them to the actual values from the validation set

In [ ]:
train, val=custom_valid_split(exp)

In [ ]:
X_train,y_cas_train, y_reg_train=prep_train_data(train, features)
X_val,y_cas_val, y_reg_val=prep_train_data(val, features)

In [ ]:
from sklearn.preprocessing import StandardScaler
PredictorScaler=StandardScaler()
TargetVarScaler=StandardScaler()

In [ ]:
#fit Training data

#reshape regular and casual values into an array
y_cas_train=y_cas_train.values.reshape(8600,1)
y_reg_train=y_reg_train.values.reshape(8600,1)

#fit X, casual and  registered
PredictorScalerFit=PredictorScaler.fit(X_train)
TargetVarScalerFit=TargetVarScaler.fit(y_cas_train)
TargetVarScalerFit=TargetVarScaler.fit(y_reg_train)

X_t=PredictorScalerFit.transform(X_train)
y_ct=TargetVarScalerFit.transform(y_cas_train)
y_rt=TargetVarScalerFit.transform(y_reg_train)

In [ ]:
#fit validation data

y_cas_val=y_cas_val.values.reshape(2286,1)
y_reg_val=y_reg_val.values.reshape(2286,1)

PredictorScalerFit=PredictorScaler.fit(X_val)
TargetVarScalerFit=TargetVarScaler.fit(y_cas_val)
TargetVarScalerFit=TargetVarScaler.fit(y_reg_val)

X_v=PredictorScalerFit.transform(X_val)
y_cv=TargetVarScalerFit.transform(y_cas_val)
y_rv=TargetVarScalerFit.transform(y_reg_val)

buid model

In [ ]:
model = Sequential()

In [ ]:
#neural network with 2 layers, one using relu and one using tanh
#learning rate is 0.01
#loss function is MSLE

model.add(Dense(units=5, input_dim=12, kernel_initializer='normal', activation='relu'))
model.add(Dense(units=5, kernel_initializer='normal', activation='relu'))
model.add(Dense(1, kernel_initializer='normal'))
opt = SGD(lr=0.01, momentum=0.9)
model.compile(loss='mean_squared_logarithmic_error', optimizer=opt, metrics=['mse'])

fit model to casual rider data

In [ ]:
model.fit(X_t, y_ct ,batch_size = 20, epochs = 50, verbose=1)

Epoch 1/50
430/430 [==============================] - 1s 1ms/step - loss: 0.0137 - mse: 2.6951
Epoch 2/50
430/430 [==============================] - 1s 1ms/step - loss: 0.0136 - mse: 2.6960
Epoch 3/50
430/430 [==============================] - 1s 1ms/step - loss: 0.0134 - mse: 2.6933
Epoch 4/50
430/430 [==============================] - 1s 1ms/step - loss: 0.0119 - mse: 2.6711
Epoch 5/50
430/430 [==============================] - 1s 1ms/step - loss: 0.0093 - mse: 2.5656
Epoch 6/50
430/430 [==============================] - 1s 1ms/step - loss: 0.0088 - mse: 2.4631
Epoch 7/50
430/430 [==============================] - 1s 1ms/step - loss: 0.0086 - mse: 2.4227
Epoch 8/50
430/430 [==============================] - 1s 1ms/step - loss: 0.0085 - mse: 2.4087
Epoch 9/50
430/430 [==============================] - 1s 1ms/step - loss: 0.0084 - mse: 2.3863
Epoch 10/50
430/430 [==============================] - 1s 1ms/step - loss: 0.0080 - mse: 2.3902
Epoch 11/50
430/430 [============================

optimize casual rider hyperparameters

In [ ]:
ResultsData=FunctionFindBestParams(X_t, y_ct, X_v, y_cv)

In [ ]:
%matplotlib inline
ResultsData.plot(x='Parameters', y='Accuracy', figsize=(15,4), kind='line')

Optimal hyperparameters are 10 batches and 50 epochs

refit model using those parameters

In [ ]:
model.fit(X_t, y_ct ,batch_size = 10, epochs = 50, verbose=1)

fit model to registered riders

In [ ]:
model2 = Sequential()

In [ ]:
model2.add(Dense(units=5, input_dim=12, kernel_initializer='normal', activation='relu'))
model2.add(Dense(units=5, kernel_initializer='normal', activation='tanh'))
model2.add(Dense(1, kernel_initializer='normal'))
opt = SGD(lr=0.01, momentum=0.9)
model2.compile(loss='mean_squared_logarithmic_error', optimizer=opt, metrics=['mse'])

In [ ]:
model2.fit(X_t, y_rt ,batch_size = 20, epochs = 50, verbose=1)

optimize registered rider hyperparameters

In [ ]:


ResultsData2=FunctionFindBestParams(X_t, y_rt, X_v, y_rv)

Optimal Hyperparameters are 20 batches and 50 epochs

In [ ]:
%matplotlib inline
ResultsData2.plot(x='Parameters', y='Accuracy', figsize=(15,4), kind='line')

No need to refit since these were the original hyperparameters

Test model

In [ ]:
#predict casual and registered riders

cas_pred=model.predict(X_v)
reg_pred=model2.predict(X_v)

In [ ]:
#undo transformation and undo log

cas_pred=np.exp(TargetVarScalerFit.inverse_transform(cas_pred))-1
reg_pred=np.exp(TargetVarScalerFit.inverse_transform(reg_pred))-1

In [ ]:
#combine predictions

pred_comb=np.round(cas_pred+reg_pred)
pred_comb[pred_comb<0]=0

In [ ]:
#retrieve actual values

actual_comb = np.exp(y_cas_val) + np.exp(y_reg_val) - 2

####Results

In [ ]:
#predict
#this sucks

get_rmsle(pred_comb, actual_comb)

1.2248682197111667

In [ ]:
rmse(pred_comb, actual_comb)

110.66753628346308

After all of that work getting a neural network to run, it looks like the random forest regressor is still far more effective. It is possible that I did the neural network wrong, but I am fairly confident I did all the steps correctly, so Random forest is superior

###Gradient Boost

In [ ]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

In [ ]:
params = {'n_estimators': 150, 'max_depth': 5, 'random_state': 0, 'min_samples_leaf' : 10, 'learning_rate': 0.1, 'subsample': 0.7, 'loss': 'ls'}
gbm_model = GradientBoostingRegressor(**params)

In [ ]:
features2 = [
    'weather', 'temp', 'atemp', 'humidity', 'windspeed',
    'holiday', 'workingday', 'season',
    'hour', 'dow', 'year', 'ideal'
]

In [ ]:
train_data, valid_data=custom_valid_split(exp)

In [ ]:
X_train,y_cl_train, y_rl_train=prep_train_data(train_data, features2)
X_val,y_cl_val, y_rl_val=prep_train_data(valid_data, features2)

In [ ]:
gbm_model.fit(X_train, y_cl_train)

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:290: FutureWarning: The loss 'ls' was deprecated in v1.0 and will be removed in version 1.2. Use 'squared_error' which is equivalent.
  FutureWarning,


GradientBoostingRegressor(loss='ls', max_depth=5, min_samples_leaf=10,
                          n_estimators=150, random_state=0, subsample=0.7)

In [ ]:
gbm_model2 = GradientBoostingRegressor(**params)
gbm_model2.fit(X_train, y_rl_train)

/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_gb.py:290: FutureWarning: The loss 'ls' was deprecated in v1.0 and will be removed in version 1.2. Use 'squared_error' which is equivalent.
  FutureWarning,


GradientBoostingRegressor(loss='ls', max_depth=5, min_samples_leaf=10,
                          n_estimators=150, random_state=0, subsample=0.7)

In [ ]:
cas_preds=np.exp(gbm_model.predict(X_val))-1
reg_preds=np.exp(gbm_model2.predict(X_val))-1

In [ ]:
comb=np.round(cas_pred+reg_pred)
comb[comb<0]=0

In [ ]:
actual_comb = np.exp(y_cl_val) + np.exp(y_rl_val) - 2

In [ ]:
actual_comb.values

array([ 39.,  23.,  16., ..., 168., 129.,  88.])

####Results

In [ ]:
get_rmsle(comb, actual_comb.values)

1.6721735839972827

In [ ]:
rmse(comb, actual_comb.values)

237.79369374487146